In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from itertools import chain
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.current_device()

0

# Prepare dataset

In [90]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [91]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [92]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [93]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [94]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['elle a certainement plus de quarante ans .', 'she is certainly above forty .']


# Set up model: encoder and decoder

In [95]:
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, device):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.device = device
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    # input shape: (B, largest seq length)
    # input is padded
    # hidden shape: (B, hidden_size)
    # outputs shape: (B, sequence length, hidden_size)
    def forward(self, inputs, hidden, batch_size=1, input_lengths=None):
        embedded = self.embedding(inputs).view(batch_size, -1, self.hidden_size)
        # in training
        if input_lengths:
            packed_input = pack_padded_sequence(embedded, input_lengths,
                                            batch_first=True, enforce_sorted=False)
            outputs, last_hidden = self.gru(packed_input, hidden)
        # in prediction
        else:
            outputs, last_hidden = self.gru(embedded, hidden)
        return last_hidden

    def initHidden(self, batch_size=1):
        return torch.zeros(1, batch_size, self.hidden_size, device=self.device)

In [96]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, device):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.device=device
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim=1)

    # input shape: (B, largest seq length)
    # input is padded
    # embedded shape: (B, seq length, hidden size)
    # outputs shape: (B, seq length, output size)
    def forward(self, inputs, hidden, batch_size=1, input_lengths=None):
        embedded = self.embedding(inputs).view(batch_size, -1, self.hidden_size)
        embedded = F.relu(embedded)
        # in training
        if input_lengths:
            packed_input = pack_padded_sequence(embedded, input_lengths,
                                           batch_first=True, enforce_sorted=False)
            outputs, hidden = self.gru(packed_input, hidden)
            outputs, output_lengths = pad_packed_sequence(outputs, batch_first=True)
        # in prediction
        else:
            outputs, hidden = self.gru(embedded, hidden)
            output_lengths = None
        outputs = self.out(outputs)
        #output = self.softmax(self.out(output[0]))
        return outputs, output_lengths

    def initHidden(self, batch_size=1):
        return torch.zeros(1, batch_size, self.hidden_size, device=self.device)

# Data generator

In [97]:
# dataset for translation
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def pad_sequence_seq2seq(batch):
    # batch is a list of data
    input_tokens = [pair[0] for pair in batch]
    output_tokens = [pair[1] for pair in batch]
    input_tokens_padded = pad_sequence(input_tokens, batch_first=True)
    input_lengths = [len(pair[0]) for pair in batch]
    output_lengths = [len(pair[1]) for pair in batch]
    output_tokens_padded = pad_sequence(output_tokens, batch_first=True)
    return input_tokens_padded, input_lengths, output_tokens_padded, output_lengths

class dataset(Dataset):
    def __init__(self, device):
        super(dataset, self).__init__()
        self.data = pairs
        self.device = device
    
    def __len__(self):
        return len(pairs)
    
    def __getitem__(self, idx):
        input, output = pairs[idx]
        input_token = torch.tensor([input_lang.word2index[word] for word in input.split()] +
                                   [EOS_token],
                                   device=self.device)
        output_token = torch.tensor([SOS_token] + 
                                    [output_lang.word2index[word] for word in output.split()] +
                                    [EOS_token],
                                    device=self.device)
        return input_token, output_token
    
data = dataset(device)
dataloader = DataLoader(data, collate_fn=pad_sequence_seq2seq ,batch_size=5)

# Construct model class

In [103]:
from torch.nn.utils.rnn import pack_padded_sequence
class seq2seq():
    def __init__(self, input_size, hidden_size, output_size, device):
        super(seq2seq, self).__init__()
        self.device = device
        self.encoder = EncoderRNN(input_size, hidden_size, self.device).to(self.device)
        self.decoder = DecoderRNN(hidden_size,output_size, self.device).to(self.device)
    def train(self, dataloader, epochs=5, encoder_lr=0.01, decoder_lr=0.01):
        criterion = nn.CrossEntropyLoss()
        encoder_optimizer = torch.optim.SGD(self.encoder.parameters(), lr=0.01)
        decoder_optimizer = torch.optim.SGD(self.decoder.parameters(), lr=0.01)
        for epoch in range(epochs):
            epoch_loss = 0
            for token_fra, token_fra_len, token_eng, token_eng_len in dataloader:
                batch_size = len(token_fra_len)
                # start running encoder
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()
                loss = 0
                # init encoder hidden
                encoder_hidden = self.encoder.initHidden(batch_size)
                # pass input senteces through encoder
                encoder_hidden = self.encoder.forward(token_fra,
                                                      encoder_hidden,
                                                    input_lengths=token_fra_len,
                                                    batch_size=batch_size
                                                    )
                # encoder ouput is first hidden of decoder
                decoder_hidden = encoder_hidden
                # decoder_outputs shape: (B, largest seq lengths, output vocab size)
                decoder_outputs, output_lengths = self.decoder.forward(token_eng,
                                                                       decoder_hidden,
                                                                       batch_size=batch_size,
                                                                      input_lengths=token_eng_len)
                
                for batch_index in range(batch_size):
                    output_length = output_lengths[batch_index]
                    loss += criterion(decoder_outputs[batch_index,0:output_length-1,:],
                             token_eng[batch_index, 1:output_length])
                    
                
                epoch_loss += loss
                loss.backward()
                decoder_optimizer.step()
                encoder_optimizer.step()
            print("epoch:", epoch, epoch_loss/len(dataloader))
    # data is a encoded sentence like nn.tensor([13,24,56,...])
    def predict(model, inputs):
        with torch.no_grad():
            
            encoder_hidden = model.encoder.initHidden(1)
            loss = 0
            print(inputs, encoder_hidden)
            encoder_hidden = model.encoder.forward(inputs,encoder_hidden)
            decoder_hidden = encoder_hidden
            decoder_output = None
            eng_output = "SOS"
            decoder_token = torch.tensor([SOS_token], device=device)
    
            while eng_output != "EOS":
            
                decoder_output,decoder_hidden = model.decoder.forward(decoder_token,
                                                                     decoder_hidden)
                decoder_token = torch.argmax(decoder_output)
                eng_output = output_lang.index2word[int(decoder_token.cpu().numpy())]
                print(eng_output)
    def save(self, model_path):
        encoder_path = model_path + "_encoder"
        decoder_path = model_path + "_decoder"
        torch.save(self.encoder.state_dict(), encoder_path)
        torch.save(self.decoder.state_dict(), decoder_path)
    
    def load(self, model_path):
        self.encoder.load_state_dict(torch.load(model_path + "_encoder"))
        self.decoder.load_state_dict(torch.load(model_path + "_decoder"))

In [104]:
model = seq2seq(input_size=input_lang.n_words, hidden_size=5,output_size=output_lang.n_words, device=device)

In [105]:
import time
start = time.time()
model.train(dataloader,epochs=1)
end = time.time()
print(end-start)

epoch: 0 tensor(24.6922, device='cuda:0', grad_fn=<DivBackward0>)
77.4280641078949


In [57]:
model.save("seq2seq")

# Make prediction

In [67]:
def predict(model, inputs):
    with torch.no_grad():
            
        encoder_hidden = model.encoder.initHidden(1)
        loss = 0
        print(inputs, encoder_hidden)
        encoder_hidden = model.encoder.forward(inputs,encoder_hidden)
        decoder_hidden = encoder_hidden
        decoder_output = None
        eng_output = "SOS"
        decoder_token = torch.tensor([SOS_token], device=device)
    
        while eng_output != "EOS":
            
            decoder_output,decoder_hidden = model.decoder.forward(decoder_token,
                                                                     decoder_hidden)
            decoder_token = torch.argmax(decoder_output)
            eng_output = output_lang.index2word[int(decoder_token.cpu().numpy())]
            print(eng_output)

In [108]:
#model = seq2seq(input_size=input_lang.n_words, hidden_size=5,output_size=output_lang.n_words, device=device)
#model.load("seq2seq")
input_sentence = "je suis mannequin ."
input_token = torch.tensor([input_lang.word2index[word] for word in input_sentence.split()] +
                                   [EOS_token],
                                   device=device)
#predict(model, input_token)
model.predict(input_token)

tensor([  6,  11, 746,   5,   1], device='cuda:0') tensor([[[0., 0., 0., 0., 0.]]], device='cuda:0')
i
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.

KeyboardInterrupt: 

# seq2seq with attention